## Совместная фильтрация элемент-элемент
1. Оффлайн часть
1.1. Получаем оценки всех пользователей
1.2. Нормализуем их (столбец mean)
1.3. Заменяем id пользователей и фильмов на категории (например фильм с id 100111 получает категорию 0, фильм c id 100112 получает категорию 1 и так далее)
1.4. Объединяем колонку mean, колонку категорий пользователей и колонку категорий фильм в разряженную матрицу coo
1.5. Создаём матрицу перекрытия, чтобы исключить фильмы у которых мало оценок. Матрица перекрытия показывает сколько пользователей одновременно оценили фильм x и фильм y. 
1.6. Получаем матрицу сходства между фильмами, для вычисления сходство используется косиносовое сходство. cor = cosine_similarity
1.7. В матрице сходства обнуляем ячейки у которых низкое сходство и укоторых маленькое перекрытие
1.8. Из полученной матрице сходств формируем таблицу сходств фильмов (similarity_table), в которой категории фильмов заменены на их настоящие id 
2. Онлайн часть
2.1. Получаем оценки текущего пользователя
2.2. Сортируем оценки по самым высоким
2.3. Вычисляем среднюю оценку пользователя (current_user_mean)
2.4. Из таблицы сходств получаем только те фильмы которые оценены текущим пользователем
2.5. Из таблицы сходств исключаем фильмы которые оценены текущим пользователем и находятся в колонки target_id. То есть мы не будем рекомендовать пользователю фильмы, которые он уже посмотрел.
2.6. Вычисляем рекомендации для пользователя, и получаем таблицу вида: target_id; prediction;sim_movies;
где target_id - id фильма рекомендуемого пользователю; prediction - ожидаемая оценка данному фильму; sim_movies- на основание каких фильмов оцененных пользователем ранее выдана данная рекомендация.

In [27]:
import requests
import time
import pandas as pd
import surprise as surprise
import numpy as np
from decimal import Decimal
from scipy.sparse import coo_matrix
from scipy import sparse
from tqdm import tqdm
# косинусовое сходство
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
users_ratings = pd.read_csv("ratings.csv")
print(users_ratings.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [29]:
users_ratings = users_ratings.drop('timestamp', axis=1)
# movie_data = pd.merge(ratings_data, movie_names, on='movieId')
users_ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [30]:
users_ratings['rating'].isnull().any()

False

In [31]:
# Нормализуем оценки вычтя из оценки среднюю оценку пользователя
def normalize(x):
    x = x.astype(float)
    x_sum = x.sum()
    x_num = x.astype(bool).sum()
    x_mean = x_sum / x_num

    if x_num == 1 or x.std() == 0:
        return 0.0
    print(x)
    print(x_mean)
    print((x.max() - x.min()))
    return (x - x_mean) / (x.max() - x.min())

In [32]:
count_col = users_ratings.groupby(['userId'])['rating'].count()
std_col = users_ratings.groupby(['userId'])['rating'].std()
mean_col = users_ratings.groupby(['userId'])['rating'].mean()
max_col = users_ratings.groupby(['userId'])['rating'].max()
min_col = users_ratings.groupby(['userId'])['rating'].min()


for index, row in users_ratings.iterrows():
    users_ratings.at[index,'mean']= 0.0 if (count_col.at[row['userId']]==1) or (std_col.at[row['userId']]==0)  else(row['rating']-mean_col.at[row['userId']])/(max_col.at[row['userId']]-min_col.at[row['userId']])
    
users_ratings.head()

,userId,movieId,rating,mean
0,1,1,4.0,-0.091595
1,1,3,4.0,-0.091595
2,1,6,4.0,-0.091595
3,1,47,5.0,0.158405
4,1,50,5.0,0.158405


In [33]:
# проверяем, что нет пропусков
users_ratings['mean'].isnull().any()

False

In [34]:
users_values = users_ratings['userId'].sort_values().unique()
movies_values = users_ratings['movieId'].sort_values().unique()
# print(movies_values)
user_categories = dict(enumerate(users_ratings['userId'].sort_values().unique()))
movie_categories = dict(enumerate(users_ratings['movieId'].sort_values().unique()))
# print(movie_categories)
for index, row in users_ratings.iterrows():
    # np.where return index   
    users_ratings.at[index,'movieId'] = np.where(movies_values == row['movieId'])[0]
    users_ratings.at[index,'userId'] = np.where(users_values == row['userId'])[0]

In [35]:
coo = coo_matrix((users_ratings['mean'], (users_ratings['movieId'],users_ratings['userId'])))

## Матрица перекрытия
Матрица перекрытия показывает сколько пользователей одновременно оценили фильм x и фильм y. 
Например цифра 3 показывает сколько людей одновременно оценили Храброе сердце и Эйс-Вентура.
Матрица смимметричная.
min_overlap=4
Мы можем не учитывать элементы, у которых мало оценок (например <= 4), так как это можно привести к неправильному значению сходства между этими фильмами.
number_of_overlaps - показывает сколько элементов осталось после примененения матрицы перекрытия

In [36]:
overlap_matrix = coo.astype(bool).astype(int).dot(coo.transpose().astype(bool).astype(int))
min_overlap=4
number_of_overlaps = (overlap_matrix > min_overlap).count_nonzero()

In [37]:
overlap_matrix.toarray()

array([[215,  68,  32, ...,   0,   0,   0],
       [ 68, 110,  26, ...,   0,   0,   0],
       [ 32,  26,  52, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   1,   1,   0],
       [  0,   0,   0, ...,   1,   1,   0],
       [  0,   0,   0, ...,   0,   0,   1]], dtype=int32)

In [38]:
M=coo.toarray()
print(M.shape)

def calculate_otiai_distance_cor(M):
    Res = np.empty((M.shape[0],M.shape[0]))
    def my_otiai_distance(i,j):
        i_mean = np.sum(M[i,:])/(M[i,:]!=0).sum()
 
        j_mean = np.sum(M[j,:])/(M[j,:]!=0).sum() 

        i_minus_i_mean =  np.where((M[i,:]!=0), M[i,:]-i_mean, 0)
     
        j_minus_j_mean =  np.where((M[j,:]!=0), M[j,:]-j_mean, 0)
  
        numerator = np.sum(i_minus_i_mean*j_minus_j_mean)

        denumerator = (np.sqrt(np.sum(np.square(i_minus_i_mean)))*np.sqrt(np.sum(np.square(j_minus_j_mean))))
        return numerator/denumerator
    for i in tqdm(range(M.shape[0])):
        for j in range(i,M.shape[0]):
            calculated = my_otiai_distance(i,j);
            if (i==j):
                Res[i,j]=calculated
            else:
                Res[i,j]=Res[j,i]=calculated
    Res[np.isnan(Res)] = 0
    return sparse.csr_matrix(Res)

cor = calculate_otiai_distance_cor(M)
print(cor.toarray())

  0%|                                                                                         | 0/9724 [00:00<?, ?it/s]

(9724, 610)


<ipython-input-38-1e633a9e9e40>:18: RuntimeWarning: invalid value encountered in double_scalars
  return numerator/denumerator
<ipython-input-38-1e633a9e9e40>:9: RuntimeWarning: invalid value encountered in double_scalars
  j_mean = np.sum(M[j,:])/(M[j,:]!=0).sum()
 22%|█████████████████                                                             | 2122/9724 [11:14<37:55,  3.34it/s]<ipython-input-38-1e633a9e9e40>:7: RuntimeWarning: invalid value encountered in double_scalars
  i_mean = np.sum(M[i,:])/(M[i,:]!=0).sum()
100%|██████████████████████████████████████████████████████████████████████████████| 9724/9724 [29:29<00:00,  5.50it/s]


[[ 1.          0.00278545  0.10441196 ...  0.          0.
   0.        ]
 [ 0.00278545  1.         -0.0085792  ...  0.          0.
   0.        ]
 [ 0.10441196 -0.0085792   1.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [39]:
def save_full_sims(cor,movies_categories):
    coo = coo_matrix(cor)
    csr = coo.tocsr()
    
    no_saved = 0
    xs, ys = coo.nonzero()

    columns = ['source_id', 'target_id', 'similarity']
    M = pd.DataFrame([],columns=columns)

    for x, y in tqdm(zip(xs, ys)):
    #     x это первый элемент кортежа, y второй; x,y это id строк описаний между которыми вычисляется сходство
    
        sim = float(csr[x, y])
        M.loc[M.shape[0]] = [int(movies_categories[x]),int(movies_categories[y]),sim]
       
        no_saved += 1
   
    M['source_id'] =  M['source_id'].astype(int)
    M['target_id'] =  M['target_id'].astype(int)
    return M

In [40]:
# full_similarity_table = save_full_sims(cor,movie_categories)
# print(full_similarity_table)

In [41]:
# full_similarity_table.to_csv('full_similarity_table_otiai.csv',index=False)

In [42]:
min_sim=0.2
# Отбрасываем элементы с низким сходством (-1 фильмы совершенно непохожи, 1 фильмы одинаковы)
cor = cor.multiply(cor > min_sim)
cor = cor.multiply(overlap_matrix > min_overlap)
print(cor.toarray())

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [43]:
np.count_nonzero(cor.toarray() > 0.2)

57172

In [44]:
# movie_categories

In [44]:
def save_sims(cor,movies_categories):
    coo = coo_matrix(cor)
    csr = coo.tocsr()
    
    no_saved = 0
    min_sim=0.2
    xs, ys = coo.nonzero()
    print(xs.shape)

    columns = ['source_id', 'target_id', 'similarity']
    M = pd.DataFrame([],columns=columns)

    for x, y in tqdm(zip(xs, ys)):
    #     x это первый элемент кортежа, y второй; x,y это id строк описаний между которыми вычисляется сходство
    
    # нет смысл сравнить строку описания с самой собой это вернёт 1 или близко к этому
        if x == y:
            continue
        sim = float(csr[x, y])
        if sim < min_sim:
            continue
        M.loc[M.shape[0]] = [int(movies_categories[x]),int(movies_categories[y]),sim]
       
        no_saved += 1
   
    M['source_id'] =  M['source_id'].astype(int)
    M['target_id'] =  M['target_id'].astype(int)
    return M

In [45]:
similarity_table = save_sims(cor,movie_categories)
print(similarity_table)

244it [00:00, 1219.91it/s]

(57172,)


57172it [01:11, 795.07it/s] 

       source_id  target_id  similarity
0              1        158    0.200086
1              1        588    0.260310
2              1       1367    0.204538
3              1       2384    0.212037
4              1       3114    0.285841
...          ...        ...         ...
53519     187593     122912    0.669746
53520     187593     122916    0.267269
53521     187595     122900    0.343527
53522     187595     122926    0.259677
53523     187595     179819    0.258464

[53524 rows x 3 columns]


In [46]:
similarity_table.to_csv('similarity_table_otiai.csv',index=False)


In [47]:
#онлайн часть

In [48]:
similarity_table = pd.read_csv("similarity_table_otiai.csv")
print(similarity_table)

       source_id  target_id  similarity
0              1        158    0.200086
1              1        588    0.260310
2              1       1367    0.204538
3              1       2384    0.212037
4              1       3114    0.285841
...          ...        ...         ...
53519     187593     122912    0.669746
53520     187593     122916    0.267269
53521     187595     122900    0.343527
53522     187595     122926    0.259677
53523     187595     179819    0.258464

[53524 rows x 3 columns]


In [49]:
# получаем оценки текущего пользователя
current_user_ratings  = users_ratings[users_ratings['userId']==1]

In [50]:
current_user_mean = current_user_ratings['rating'].mean()
print(current_user_mean)
# сортируем оценки по самым высоким
current_user_ratings.sort_values(['rating'], ascending=[0],inplace=True)
current_user_ratings.reset_index(drop=True, inplace=True)
print(current_user_ratings)

3.9482758620689653
    userId  movieId  rating      mean
0        1     8810     5.0  0.350575
1        1     8663     5.0  0.350575
2        1     8287     5.0  0.350575
3        1     7679     5.0  0.350575
4        1     6784     5.0  0.350575
5        1     7419     5.0  0.350575
6        1     1283     4.5  0.183908
7        1     6693     4.5  0.183908
8        1     6993     4.5  0.183908
9        1     7398     4.5  0.183908
10       1      291     4.0  0.017241
11       1     8448     4.0  0.017241
12       1     7572     4.0  0.017241
13       1     7355     4.0  0.017241
14       1     7241     4.0  0.017241
15       1     6298     4.0  0.017241
16       1     6236     4.0  0.017241
17       1     4607     4.0  0.017241
18       1     2670     4.0  0.017241
19       1     7750     3.5 -0.149425
20       1     8045     3.5 -0.149425
21       1     5294     3.5 -0.149425
22       1     8532     3.5 -0.149425
23       1     7306     3.0 -0.316092
24       1     7137     3.0 -0.

<ipython-input-50-7dec9e58f173>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_user_ratings.sort_values(['rating'], ascending=[0],inplace=True)


In [51]:
# получаем только те фильмы которые оцнены текущим пользователем и находятся в таблице сходств
in_source=similarity_table['source_id'].isin(current_user_ratings['movieId']) 
# Исключаем из таблице сходств уже оцененные фильмы пользователи (не нужно рекомендовать, то, что уже было просмотренно)
not_in_target=~similarity_table['target_id'].isin(current_user_ratings['movieId'])

# Получаем таблиц сходств для текущего пользователя
top_by_sims = similarity_table.loc[in_source &  not_in_target].sort_values(['similarity'], ascending=[0])[:30]
top_by_sims.reset_index(drop=True, inplace=True)
print(top_by_sims)

    source_id  target_id  similarity
0        8810      31221    0.596036
1        8810       7439    0.574648
2        7137       3030    0.475561
3        7137       1343    0.470132
4        8810      27839    0.467656
5        7137       8781    0.463111
6        7137       6618    0.446859
7        6993       1962    0.445127
8        8810       6754    0.435469
9        7137       6059    0.416325
10       8810       6294    0.402908
11       6993       5617    0.388103
12       7137       3317    0.383484
13       8810      36708    0.379379
14       8810      37386    0.378222
15       1283       3334    0.372872
16       8810       7324    0.372808
17       8810      32029    0.372560
18       8810       4887    0.358948
19       7137       8983    0.354824
20       1283       2921    0.349602
21       5294       8366    0.349194
22       8810       4876    0.341923
23        277        616    0.341620
24       8810       8781    0.332706
25       7137       3146    0.330645
2

In [52]:
columns = ['target_id', 'prediction', 'sim_movies']
recs = pd.DataFrame([],columns=columns)
targets = top_by_sims['target_id']

for target_row in targets:
    pre = 0
    sim_sum = 0

    rated_items = top_by_sims[top_by_sims['target_id']==target_row]

    if len(rated_items) > 0:
        for index, sim_item in rated_items.iterrows():
#             получаем оценку пользователя для фильма
            user_rating = current_user_ratings[current_user_ratings['movieId']==sim_item['source_id']].iloc[0]['rating']
#         вычитаем из оценки среднюю для пользователя
            r = user_rating- current_user_mean
#     умножаем схожесть на оценку
            pre += sim_item.similarity * r
            sim_sum += sim_item.similarity
        if sim_sum > 0:
# формируем прогноз
            recs.loc[recs.shape[0]] = [target_row,current_user_mean + pre/sim_sum,rated_items['source_id'].values]

recs.sort_values(['prediction'], ascending=[0],inplace=True)
recs.reset_index(drop=True, inplace=True)
print(recs)


   target_id  prediction    sim_movies
0      31221    5.000000        [8810]
1      36708    5.000000        [8810]
2      37380    5.000000        [8810]
3       6541    5.000000        [8810]
4       4876    5.000000        [8810]
5       4887    5.000000        [8810]
6      32029    5.000000        [8810]
7       7324    5.000000        [8810]
8       7439    5.000000        [8810]
9      37386    5.000000        [8810]
10      5528    5.000000        [8810]
11      6294    5.000000        [8810]
12     27839    5.000000        [8810]
13      6754    5.000000        [8810]
14      5617    4.500000        [6993]
15      1962    4.500000        [6993]
16      2921    4.500000        [1283]
17      3334    4.500000        [1283]
18      8781    3.836137  [7137, 8810]
19      8781    3.836137  [7137, 8810]
20      8366    3.500000        [5294]
21      6618    3.000000        [7137]
22      8983    3.000000        [7137]
23      3317    3.000000        [7137]
24      6059    3.000000 

In [53]:
def get_recs(userId, n=30):
    current_user_ratings  = users_ratings[users_ratings['userId']==userId]
    
    current_user_mean = current_user_ratings['rating'].mean()

    # сортируем оценки по самым высоким
    current_user_ratings.sort_values(['rating'], ascending=[0],inplace=True)
    current_user_ratings.reset_index(drop=True, inplace=True)
    
    # получаем только те фильмы которые оцнены текущим пользователем и находятся в таблице сходств
    in_source=similarity_table['source_id'].isin(current_user_ratings['movieId']) 
    # Исключаем из таблице сходств уже оцененные фильмы пользователи (не нужно рекомендовать, то, что уже было просмотренно)
    not_in_target=~similarity_table['target_id'].isin(current_user_ratings['movieId'])

    # Получаем таблиц сходств для текущего пользователя
    top_by_sims = similarity_table.loc[in_source &  not_in_target].sort_values(['similarity'], ascending=[0])[:n]
    top_by_sims.reset_index(drop=True, inplace=True)
    
    columns = ['target_id', 'prediction', 'sim_movies']
    recs = pd.DataFrame([],columns=columns)
    targets = top_by_sims['target_id']

    for target_row in targets:
        pre = 0
        sim_sum = 0

        rated_items = top_by_sims[top_by_sims['target_id']==target_row]

        if len(rated_items) > 0:
            for index, sim_item in rated_items.iterrows():
    #             получаем оценку пользователя для фильма
                user_rating = current_user_ratings[current_user_ratings['movieId']==sim_item['source_id']].iloc[0]['rating']
    #         вычитаем из оценки среднюю для пользователя
                r = user_rating- current_user_mean
    #     умножаем схожесть на оценку
                pre += sim_item.similarity * r
                sim_sum += sim_item.similarity
            if sim_sum > 0:
    # формируем прогноз
                recs.loc[recs.shape[0]] = [target_row,current_user_mean + pre/sim_sum,rated_items['source_id'].values]

    recs.sort_values(['prediction'], ascending=[0],inplace=True)
    recs.reset_index(drop=True, inplace=True)
    
    return recs
    

In [54]:
get_recs(1, n=30)

<ipython-input-53-e3105631a65a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_user_ratings.sort_values(['rating'], ascending=[0],inplace=True)


,target_id,prediction,sim_movies
0,31221,5.000000,[8810]
1,36708,5.000000,[8810]
2,37380,5.000000,[8810]
3,6541,5.000000,[8810]
4,4876,5.000000,[8810]
5,4887,5.000000,[8810]
6,32029,5.000000,[8810]
7,7324,5.000000,[8810]
8,7439,5.000000,[8810]
9,37386,5.000000,[8810]
